# Problem 2: Visualize an interactive map (10 points)

In [38]:
# load modules
import geopandas as gpd
import pandas as pd
from pyproj import CRS
import matplotlib.pyplot as plt
import mapclassify
import contextily as ctx
import folium
%matplotlib inline

In [39]:
# Create a Map instance
m1 = folium.Map(location=[65.25, 26.8], zoom_start=5, control_scale=True, tiles='cartodbpositron')

In [40]:
# Read population data of municipality
wfs = 'http://geo.stat.fi/geoserver/vaestoalue/wfs?service=WFS&request=GetFeature&typename=vaestoalue:kunta_vaki2018&outputformat=JSON'
pop = gpd.read_file(wfs)

In [41]:
# Read traffic injuries data 
wfs2 = 'http://geo.stat.fi/geoserver/tieliikenne/wfs?service=WFS&request=GetFeature&version=1.0.0&typename=tieliikenne:tieliikenne_2018'
traffic = gpd.read_file(wfs2)

In [42]:
# Check the crs and set the right crs
print(traffic.crs)
print(pop.crs)

None
epsg:3067


In [43]:
# Set the right crs
traffic.crs = CRS.from_epsg(3067)

In [44]:
# Check the crs and set the right crs
traffic.crs==pop.crs

True

In [45]:
# Project them both to 4326 wgs84
pop = pop.to_crs(epsg=4326)
traffic = traffic.to_crs(epsg=4326)

In [46]:
# Get x and y coordinates for each points
traffic['x'] = traffic['geometry'].x
traffic['y'] = traffic['geometry'].y

# Create a list of coordinate pairs
locations = list(zip(traffic['y'],traffic['x']))

In [47]:
# Heatmap 
#from folium.plugins import HeatMap

# Create a Map instance
#m = folium.Map(location=[60.25, 24.8], tiles = 'stamentoner', zoom_start=10, control_scale=True)

# Add heatmap to map instance
# Available parameters: HeatMap(data, name=None, min_opacity=0.5, max_zoom=18, max_val=1.0, radius=25, blur=15, gradient=None, overlay=True, control=True, show=True)
#HeatMap(locations).add_to(m)

# Alternative syntax:
#m.add_child(HeatMap(points_array, radius=15))

# Show map
#m

In [48]:
pop['geoid'] = pop.index.astype(str)

In [49]:
pop = pop[['geoid','geometry','ika_15_64p','ika_65_p','name']]

In [50]:
pop = pop.rename(columns={'ika_15_64p':'age_15_64',
                          'ika_65_p':'age_65'})

In [51]:
# Plot a choropleth map for population aged 15-65
# Notice: 'geoid' column that we created earlier needs to be assigned always as the first column
folium.Choropleth(
    geo_data=pop,
    name='Percentage in population aged 15-65 in 2018',
    data=pop,
    columns=['geoid', 'age_15_64'],
    key_on='feature.id',
    fill_color='YlOrRd',
    fill_opacity=0.7,
    line_opacity=0.2,
    line_color='white', 
    line_weight=0,
    highlight=False, 
    smooth_factor=1.0,
    #threshold_scale=[100, 250, 500, 1000, 2000],
    legend_name= 'Percentage in population aged 15-65 in 2018').add_to(m1)

In [52]:
# Plot a choropleth map for population aged 65+
# Notice: 'geoid' column that we created earlier needs to be assigned always as the first column
#folium.Choropleth(
#    geo_data=pop,
#    name='Percentage in population aged 65+ in 2018',
#    data=pop,
#    columns=['geoid', 'age_65'],
#    key_on='feature.id',
#    fill_color='YlOrRd',
#    fill_opacity=0.7,
    line_opacity=0.2,
    line_color='white', 
    line_weight=0,
    highlight=False, 
    smooth_factor=1.0,
    #threshold_scale=[100, 250, 500, 1000, 2000],
    legend_name= 'Percentage in population aged 65+ in 2018').add_to(m1)

In [53]:
from folium.plugins import MarkerCluster

# Create a folium marker cluster for traffic accidents in 2018
marker_cluster = MarkerCluster(locations,name= 'Counts of Traffic accidents in 2018')

# Add marker cluster to map
marker_cluster.add_to(m1)


In [54]:
# Convert points to GeoJson and create the interactive polygons
folium.features.GeoJson(pop,  
                        name='Interactive polygons',
                        style_function=lambda x: {'color':'transparent','fillColor':'transparent','weight':0},
                        tooltip=folium.features.GeoJsonTooltip(fields=['name','age_15_64','age_65'],
                                                                aliases = ['Municipality: ','Population aged 15-65 (%): ','Population aged 65+ (%): '],
                                                                labels=True,
                                                                sticky=False
                                                                            )
                       ).add_to(m1)

In [55]:
#folium.features.GeoJson(pop,  
#                        name='Labels',
#                        style_function=lambda x: {'color':'transparent','fillColor':'transparent','weight':0},
#                        tooltip=folium.features.GeoJsonTooltip(fields=['age_65'],
#                                                                aliases = ['Population aged 65+ (%): '],
#                                                                labels=True,
#                                                                sticky=False
#                                                                            )
#                       ).add_to(m1)

In [56]:
# Create a layer control object and add it to our map instance
folium.LayerControl().add_to(m1)

#Show map
m1

In [20]:
outfp = 'docs/Problem2_base_map.html'
m1.save(outfp)